In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
layers = tf.keras.layers
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0  # normalize to [0,1] range
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\1\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\1\test'
checkpoint_path = r'C:\Users\24306\Desktop\MobileNetV3Large\1\MobileNetV3Large.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[39.0, 84.0, 40.0, 13.0, 80.0, 37.0, 100.0, 10.0, 100.0, 86.0, 20.0, 20.0, 100.0, 95.0, 37.0, 23.0, 84.0, 95.0, 80.0, 20.0, 95.0, 96.0, 13.0, 95.0, 45.0, 86.0, 7.0, 4.0, 84.0, 100.0, 96.0, 55.0, 95.0, 37.0, 40.0, 13.0, 30.0, 80.0, 47.0, 86.0, 45.0, 84.0, 13.0, 35.0, 32.0, 23.0, 80.0, 45.0, 16.0, 96.0, 86.0, 86.0, 96.0, 95.0, 100.0, 32.0, 47.0, 86.0, 30.0, 45.0, 84.0, 52.0, 47.0, 86.0, 96.0, 37.0, 47.0, 13.0, 40.0, 84.0, 13.0, 13.0, 37.0, 100.0, 35.0, 95.0, 13.0, 7.0, 96.0, 37.0, 45.0, 52.0, 16.0, 100.0, 16.0, 86.0, 93.0, 47.0, 47.0, 47.0, 16.0, 30.0, 4.0, 16.0, 35.0, 84.0, 66.0, 23.0, 86.0, 10.0, 66.0, 100.0, 100.0, 100.0, 16.0, 10.0, 4.0, 10.0, 7.0, 95.0, 84.0, 20.0, 20.0, 23.0, 23.0, 7.0, 47.0, 95.0, 10.0, 55.0, 37.0, 13.0, 30.0, 47.0, 84.0, 30.0, 55.0, 16.0, 66.0, 39.0, 47.0, 40.0, 93.0, 30.0, 96.0, 37.0, 16.0, 23.0, 30.0, 37.0, 47.0, 100.0, 16.0, 7.0, 10.0, 93.0, 13.0, 95.0, 93.0, 10.0, 100.0, 10.0, 37.0, 86.0, 93.0, 80.0, 13.0, 93.0, 7.0, 23.0, 47.0, 30.0, 100.0, 84.0, 80.0, 100.0

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
MobileNetV3Large = tf.keras.applications.MobileNetV3Large(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
MobileNetV3Large.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
x = MobileNetV3Large(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3large (Functiona  (None, 7, 7, 960)        2996352   
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 960)              0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 961       
                                                                 
Total params: 2,997,313
Trainable params: 2,972,913
Non-trainable params: 24,400
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
np.savetxt('MobileNetV3Large_loss_1.txt',history.history['loss'])
#np.savetxt('DenseNet201_acc.txt',history.history['acc'])
np.savetxt('MobileNetV3Large_val_loss_1.txt',history.history['val_loss'])
#np.savetxt('DenseNet201_val_acc.txt',history.history['val_acc'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Wed Apr 26 10:07:18 2023
Epoch 1/300
590/591 [============================>.] - ETA: 0s - loss: 684.8411

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\1\MobileNetV3Large.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\1\MobileNetV3Large.ckpt\assets


591/591 [==============================] - 70s 100ms/step - loss: 684.0538 - val_loss: 3832.8835
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 83.2220

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\1\MobileNetV3Large.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\1\MobileNetV3Large.ckpt\assets


591/591 [==============================] - 85s 144ms/step - loss: 0.7679 - val_loss: 0.3538
Epoch 296/300
591/591 [==============================] - ETA: 0s - loss: 0.8492

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\1\MobileNetV3Large.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\MobileNetV3Large\1\MobileNetV3Large.ckpt\assets


591/591 [==============================] - 90s 152ms/step - loss: 0.8492 - val_loss: 0.3533
Epoch 297/300
591/591 [==============================] - 67s 113ms/step - loss: 0.6761 - val_loss: 0.3562
Epoch 298/300
591/591 [==============================] - 66s 112ms/step - loss: 0.9423 - val_loss: 0.3901
Epoch 299/300
591/591 [==============================] - 67s 112ms/step - loss: 0.7200 - val_loss: 0.3589
Epoch 300/300
591/591 [==============================] - 67s 113ms/step - loss: 0.8569 - val_loss: 0.4031
end time: Wed Apr 26 15:17:21 2023
